In [1]:
import pandas as pd
import math
import numpy as np

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "valid_log.csv").is_file():
        cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
        cfg = OmegaConf.to_container(cfg)
        num_epochs = cfg['optim']['epochs']

        valid_log = pd.read_csv(run / "valid_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-3) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    len(list(Path('.').glob('*')))
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            for run in list(regime.glob("run-*")):
                metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

def collect_all_regimes(model_name, root, csv_file, regimes=['0.1', '0.2', '0.5', '1.0']):
    root = Path(root)

    metrics = []
    for run in list(root.glob("run-*")):
        if root.name.rsplit("-", 1)[1] in regimes:
            metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], t_student=False):
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate(['mean', 'std'])
    
    return mean_metrics

<h1>Evaluation - Searching temperature hyperparameter</h1>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

<h2>GlaS Dataset</h2>

In [4]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t')),
}

In [7]:
# Collect predictions scanning runs
#predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [8]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,50.85,34.68,40.09,8.25
1,H-UNet-BASE-SWTA,0,5.0,1.0,47.75,32.12,51.58,10.07
2,H-UNet-BASE-SWTA,0,10.0,1.0,47.89,32.06,48.38,8.85
3,H-UNet-BASE-SWTA,0,20.0,1.0,49.35,33.32,38.16,7.22
4,H-UNet-BASE-SWTA,0,50.0,1.0,49.69,33.65,46.86,8.39
5,H-UNet-BASE-SWTA,0,100.0,1.0,46.50,30.83,46.22,8.27
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,49.11,33.12,39.84,8.40
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,48.96,33.04,51.24,9.87
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,49.02,32.99,47.75,8.82
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,48.94,33.01,38.14,7.76


Dice     Jaccard     Hausdorff Distance  \
                                     mean std    mean std               mean   
Model              Inv Temp Regime                                             
H-UNet-BASE-SWTA   1.0      1.0     50.85 NaN   34.68 NaN              40.09   
                   5.0      1.0     47.75 NaN   32.12 NaN              51.58   
                   10.0     1.0     47.89 NaN   32.06 NaN              48.38   
                   20.0     1.0     49.35 NaN   33.32 NaN              38.16   
                   50.0     1.0     49.69 NaN   33.65 NaN              46.86   
                   100.0    1.0     46.50 NaN   30.83 NaN              46.22   
H-UNet-BASE-SWTA-T 1.0      1.0     49.11 NaN   33.12 NaN              39.84   
                   5.0      1.0     48.96 NaN   33.04 NaN              51.24   
                   10.0     1.0     49.02 NaN   32.99 NaN              47.75   
                   20.0     1.0     48.94 NaN   33.01 NaN              38.14   
                   50.0     1.0     48.50 NaN   32.68 NaN              47.86   
                   100.0    1.0     48.12 NaN   32.27 NaN              49.67   

                                       Average Surface Distance      
                                   std                     mean std  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   1.0      1.0    NaN                     8.25 NaN  
                   5.0      1.0    NaN                    10.07 NaN  
                   10.0     1.0    NaN                     8.85 NaN  
                   20.0     1.0    NaN                     7.22 NaN  
                   50.0     1.0    NaN                     8.39 NaN  
                   100.0    1.0    NaN                     8.27 NaN  
H-UNet-BASE-SWTA-T 1.0      1.0    NaN                     8.40 NaN  
                   5.0      1.0    NaN                     9.87 NaN  
                   10.0     1.0    NaN                     8.82 NaN  
                   20.0     1.0    NaN                     7.76 NaN  
                   50.0     1.0    NaN                     9.01 NaN  
                   100.0    1.0    NaN                     9.49 NaN

In [9]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    #'H-UNet-BASE-SWTA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft_fromBestDice')),
    #'H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft_fromBestDice')),
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t_ft')),
}

In [10]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

Wrong number of epochs in run: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet_base-swta_ft/inv_temp-1/regime-0.2/run-5
Skipping not found: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet_base-swta_ft/inv_temp-1/regime-0.2/run-5/test_predictions/preds.csv
Wrong number of epochs in run: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet_base-swta_ft/inv_temp-20/regime-0.2/run-0
More than 1 tensorboard folder in run: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet_base-swta_ft/inv_temp-20/regime-0.2/run-0
Wrong number of epochs in run: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet_base-swta_t_ft/inv_temp-20/regime-0.2/run-5
More than 1 tensorboard folder in run: /home/luca/dgx-a1

In [11]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT,0,1.0,0.2,79.79,67.36,24.53,3.45
1,H-UNet-BASE-SWTA-FT,0,5.0,0.2,82.07,70.66,21.62,3.31
2,H-UNet-BASE-SWTA-FT,0,10.0,0.2,80.44,68.09,23.30,2.95
3,H-UNet-BASE-SWTA-FT,0,20.0,0.2,79.81,67.19,20.98,2.78
4,H-UNet-BASE-SWTA-FT,0,50.0,0.2,84.32,73.49,13.71,1.96
5,H-UNet-BASE-SWTA-FT,0,100.0,0.2,82.88,71.59,16.67,2.34
6,H-UNet-BASE-SWTA-FT,1,1.0,0.2,77.62,64.50,26.04,3.44
7,H-UNet-BASE-SWTA-FT,1,5.0,0.2,77.02,63.82,25.33,3.34
8,H-UNet-BASE-SWTA-FT,1,10.0,0.2,78.70,65.68,26.41,3.38
9,H-UNet-BASE-SWTA-FT,1,20.0,0.2,78.41,65.76,28.14,3.91


Dice           Jaccard            \
                                       mean       std    mean       std   
Model               Inv Temp Regime                                       
H-UNet-BASE-SWTA-FT 1.0      0.2     77.908  2.273152  64.846  2.924257   
                    5.0      0.2     79.028  3.051388  66.520  3.953347   
                    10.0     0.2     79.580  1.912106  67.064  2.569860   
                    20.0     0.2     80.082  2.020587  67.826  2.675720   
                    50.0     0.2     81.040  2.851342  69.230  3.711287   
                    100.0    0.2     80.348  3.652913  68.236  4.842415   
H-UNet-BASE-SWTA-T  1.0      0.2     77.990  1.262874  65.016  1.707141   
                    5.0      0.2     80.442  2.934224  68.432  3.977106   
                    10.0     0.2     81.832  1.891261  70.186  2.611289   
                    20.0     0.2     81.310  1.784979  69.614  2.369120   
                    50.0     0.2     80.506  2.185299  68.584  2.773415   
                    100.0    0.2     81.814  1.681363  70.268  2.248982   

                                    Hausdorff Distance            \
                                                  mean       std   
Model               Inv Temp Regime                                
H-UNet-BASE-SWTA-FT 1.0      0.2                25.102  3.203298   
                    5.0      0.2                26.982  4.911840   
                    10.0     0.2                24.500  2.350979   
                    20.0     0.2                24.232  4.013816   
                    50.0     0.2                21.474  4.642599   
                    100.0    0.2                22.014  4.076902   
H-UNet-BASE-SWTA-T  1.0      0.2                26.402  5.075201   
                    5.0      0.2                24.062  3.915044   
                    10.0     0.2                23.774  4.891424   
                    20.0     0.2                23.552  3.111281   
                    50.0     0.2                24.178  5.295453   
                    100.0    0.2                20.822  3.254377   

                                    Average Surface Distance            
                                                        mean       std  
Model               Inv Temp Regime                                     
H-UNet-BASE-SWTA-FT 1.0      0.2                       3.478  0.435511  
                    5.0      0.2                       3.714  0.674003  
                    10.0     0.2                       3.270  0.188812  
                    20.0     0.2                       3.300  0.516382  
                    50.0     0.2                       2.960  0.612781  
                    100.0    0.2                       3.086  0.631965  
H-UNet-BASE-SWTA-T  1.0      0.2                       3.768  0.749713  
                    5.0      0.2                       3.314  0.512669  
                    10.0     0.2                       3.180  0.595525  
                    20.0     0.2                       3.252  0.339662  
                    50.0     0.2                       3.410  0.661740  
                    100.0    0.2                       3.012  0.522657

<h1>Evaluation - Hebbian Unsupervised Pretraining</h1>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [12]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

INV_TEMP_GlaS=10          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
}

In [13]:
# Collect predictions scanning runs
#predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
predictions = pd.concat([collect_one(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

Skipping not found: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet_base-hpca/inv_temp-1/regime-1.0/run-0/test_predictions/preds.csv
Skipping not found: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet_base-hpca_t/inv_temp-1/regime-1.0/run-0/test_predictions/preds.csv


In [14]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,10.0,1.0,47.89,32.06,48.38,8.85
1,H-UNet-BASE-SWTA-T,0,10.0,1.0,49.02,32.99,47.75,8.82


Dice     Jaccard     Hausdorff Distance  \
                                     mean std    mean std               mean   
Model              Inv Temp Regime                                             
H-UNet-BASE-SWTA   10.0     1.0     47.89 NaN   32.06 NaN              48.38   
H-UNet-BASE-SWTA-T 10.0     1.0     49.02 NaN   32.99 NaN              47.75   

                                       Average Surface Distance      
                                   std                     mean std  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   10.0     1.0    NaN                     8.85 NaN  
H-UNet-BASE-SWTA-T 10.0     1.0    NaN                     8.82 NaN

<h1>Evaluation - Data regime variations</h1>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

<h2>GlaS Dataset</h2>

In [15]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

REGIMES = ['0.2']       # regimes to be considered
INV_TEMP_GlaS=10          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=glas/unet_base/inv_temp-1').glob('regime-*')),
    #'UNet-256': list(Path(EXP_ROOT + '/experiment=glas/unet_base-256/inv_temp-1').glob('regime-*')),
    'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=glas/unet/inv_temp-1').glob('regime-*')),
    #'H-UNet-HPCA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft_fromBestDice/inv_temp-1').glob('regime-*')),
    #'H-UNet-HPCA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft_fromBestDice/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    #'H-UNet-SWTA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft_fromBestDice/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-SWTA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft_fromBestDice/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
}

In [16]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv', regimes=REGIMES) for k, v in runs.items() for r in v], ignore_index=True)

Wrong number of epochs in run: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet-hpca_ft/inv_temp-1/regime-0.2/run-0
Skipping not found: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=glas/hunet-hpca_ft/inv_temp-1/regime-0.2/run-0/test_predictions/preds.csv


In [17]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA-FT,0,1.0,0.2,79.98,67.37,20.85,2.83
1,H-UNet-HPCA-FT,1,1.0,0.2,79.25,66.79,17.15,2.71
2,H-UNet-HPCA-FT,2,1.0,0.2,77.99,64.86,34.29,4.07
3,H-UNet-HPCA-FT,3,1.0,0.2,80.78,68.85,29.26,4.05
4,H-UNet-HPCA-FT,4,1.0,0.2,80.59,68.51,24.45,3.54
5,H-UNet-HPCA-FT,5,1.0,0.2,80.01,67.74,28.63,4.30
6,H-UNet-HPCA-FT,6,1.0,0.2,81.83,70.24,20.91,3.12
7,H-UNet-HPCA-FT,7,1.0,0.2,77.65,64.76,34.50,5.04
8,H-UNet-HPCA-FT,8,1.0,0.2,81.86,70.32,22.69,2.90
9,H-UNet-HPCA-FT,9,1.0,0.2,79.51,66.82,24.46,3.20


Dice           Jaccard            \
                                       mean       std    mean       std   
Model               Inv Temp Regime                                       
H-UNet-HPCA-FT      1.0      0.2     79.945  1.418154  67.626  1.933967   
H-UNet-HPCA-T-FT    1.0      0.2     80.540  3.525450  68.601  4.589274   
H-UNet-SWTA-FT      10.0     0.2     79.580  1.912106  67.064  2.569860   
H-UNet-SWTA-T-FT    10.0     0.2     81.832  1.891261  70.186  2.611289   
UNet                1.0      0.2     78.462  2.753385  65.684  3.510575   
UNet-Pseudolabeling 1.0      0.2     80.919  3.641518  69.475  4.624633   

                                    Hausdorff Distance            \
                                                  mean       std   
Model               Inv Temp Regime                                
H-UNet-HPCA-FT      1.0      0.2                25.719  5.813894   
H-UNet-HPCA-T-FT    1.0      0.2                24.523  6.083800   
H-UNet-SWTA-FT      10.0     0.2                24.500  2.350979   
H-UNet-SWTA-T-FT    10.0     0.2                23.774  4.891424   
UNet                1.0      0.2                27.049  2.833017   
UNet-Pseudolabeling 1.0      0.2                31.897  8.037603   

                                    Average Surface Distance            
                                                        mean       std  
Model               Inv Temp Regime                                     
H-UNet-HPCA-FT      1.0      0.2                       3.576  0.763780  
H-UNet-HPCA-T-FT    1.0      0.2                       3.487  0.939622  
H-UNet-SWTA-FT      10.0     0.2                       3.270  0.188812  
H-UNet-SWTA-T-FT    10.0     0.2                       3.180  0.595525  
UNet                1.0      0.2                       3.807  0.447314  
UNet-Pseudolabeling 1.0      0.2                       4.642  1.247841